In [2]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from dateutil.parser import parse, isoparse
from dateutil.relativedelta import *
from fin_stat_names import stats as fin_stat_names

In [3]:
def get_right_df(file):
    df = pd.read_csv(file) 
    if 'dividend.csv' in file:
        date_colums = ['Ex-Date', 'Record Date', 'Pay Date', 'Declared Date']
        for col in date_colums:
            df[col] = df[col].apply(get_date)
        df.columns = ['Ex-Date', 'Record Date', 'Pay Date', 'date', 'Type', 'Amount']
        return df
    df['date'] = df['date'].apply(get_date, args=(file,))
    return df

def check(df, col):
    d = {1: 31, 10: 30, 7: 30, 4: 30}
    for date in df[df[col].notnull()].date[0:10]:
        try:
            if d[date.month] != date.day:
                return False
        except:
            return False
    return True

def check_all(df):
    for column in df.columns:
        if len(df[df[column].notnull()]) == 0:
            continue
        a = column + '.csv' in fin_stat_names
        b = check(df, column)
        if a != b:
            print(a, column)
            return False
    return True

def get_date(s, name = 'dividends'):
    try:
        flag = name.split('/')[-1] in fin_stat_names
        return parse(s) + relativedelta(months=1) * flag
    except:
        if s == '--':
            return
        print('file', name)
        print('Не удалось преобразовать строку', s)
        return

def transform_fin_date(s):
    return isoparse(s) + relativedelta(months=2) - relativedelta(days=1)

def get_true_fin(path):
    df = pd.read_csv(path)
    columns = df.T.iloc[0]
    df = df.T
    df.columns = columns
    df = df.iloc[1:]
    df = df.reset_index()
    df = df.rename(columns={'index' : 'date'})
    df.date = df.date.apply(transform_fin_date)
    return df.drop(['Restated', 'As Originally Reported'], axis=1)

def get_merged_table(ticker):
    path = '../Data/stocks/{ticker}/key_stats/'.format(ticker=ticker)
    df = pd.DataFrame(columns=['date'])
    for file in tqdm(os.listdir(path), desc=ticker):
        new_df = get_right_df(path + file)
        df = pd.merge(df, new_df, on='date', how='outer')
    df = df.sort_values('date', ascending=False)
    assert check_all(df) == True
    a = get_true_fin('../Data/stocks/{ticker}/financials_stats/balance_sheet.csv'.format(ticker=ticker))
    b = get_true_fin('../Data/stocks/{ticker}/financials_stats/cash_flow_statement.csv'.format(ticker=ticker))
    c = get_true_fin('../Data/stocks/{ticker}/financials_stats/income_statement.csv'.format(ticker=ticker))
    a = pd.merge(a, b,  on='date', how='outer')
    a = pd.merge(a, c,  on='date', how='outer')
    df = pd.merge(df, a,  on='date', how='outer')
    df['ticker'] = [ticker] * len(df)
    cols = df.columns.to_list()
    return df[[cols[0]] + [cols[-1]] + cols[1 : -1]]

In [4]:
tickers = [
    'APA',
    'COG',
    'CVX',
    'COP',
    'EOG',
    'XOM',
    'HES',
    'MUR',
    'OXY',
    'SUN',
    'VLO',
    'BA',
    'LMT',
    'AAL',
    'DAL',
    'JBLU',
    'LUV',
    'HON',
    'RTN',
    'UTX',
    'GE',
    #'FDX',
    'UPS',
    'AEP',
    'ETR',
    'PCG',
    'JE',
    'XEL',
    'ES',
    'AGR',
    'CPK',
    'NEE',
    'HE',
    'AVA',
    'IDA',
    'AEE',
    'DUK',
    'NI',
    'EXC'
]

In [5]:
res = pd.DataFrame(columns=['date'])
for ticker in tqdm(tickers):
    res = pd.concat([res, get_merged_table(ticker)], sort=False)
    print('len = ', res.shape)


len =  (12259, 264)



len =  (19959, 282)



len =  (32851, 289)



len =  (45116, 294)



len =  (52942, 298)


...


len =  (333723, 314)



len =  (340262, 314)



len =  (352519, 314)



len =  (364760, 314)



len =  (377002, 314)



In [9]:
res.to_csv('test.csv', index=False)

In [13]:
res = res.sort_values(['date', 'ticker'], ascending=False)

In [10]:
pd.read_csv('test.csv', low_memory=False)

,date,ticker,historical_daily_var_1_all,cash_financing_ttm,ev_ebitda,zmijewski_score,market_beta_60_month,market_cap_fractile,revenues_growth,ni_per_employee_annual,...,Amortization Expense,Net Change in Intangibles,Rent and Landing Expense,Notes Receivable,Properties,Excise Taxes,Research and Development Expense,Total Preferred Dividends Paid,Leases,Provision for Doubtful Accounts
0,2020-03-19,APA,NaN,NaN,175.33,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-18,APA,NaN,NaN,172.37,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-17,APA,NaN,NaN,172.49,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-16,APA,NaN,NaN,178.29,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-14,APA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376997,1972-06-06,EXC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376998,1972-06-05,EXC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376999,1972-06-02,EXC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377000,1972-06-01,EXC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
res.columns[14]

'Amount'

In [26]:
res[res.ticker == 'EXC'].iloc[-1]['Ex-Date']

Timestamp('2000-10-23 00:00:00')

In [24]:
for idx, el in enumerate(res[res.ticker == 'EXC'].iloc[-1]):
    if el:
        print(idx, el)

0 NaT
1 EXC
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 2000-10-23 00:00:00
11 NaT
12 NaT
13 Normal
14 0.0791
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
40 nan
41 nan
42 nan
43 nan
44 nan
45 nan
46 nan
47 nan
48 nan
49 nan
50 nan
51 nan
52 nan
53 nan
54 nan
55 nan
56 nan
57 nan
58 nan
59 nan
60 nan
61 nan
62 nan
63 nan
64 nan
65 nan
66 nan
67 nan
68 nan
69 nan
70 nan
71 nan
72 nan
73 nan
74 nan
75 nan
76 nan
77 nan
78 nan
79 nan
80 nan
81 nan
82 nan
83 nan
84 nan
85 nan
86 nan
87 nan
88 nan
89 nan
90 nan
91 nan
92 nan
93 nan
94 nan
95 nan
96 nan
97 nan
98 nan
99 nan
100 nan
101 nan
102 nan
103 nan
104 nan
105 nan
106 nan
107 nan
108 nan
109 nan
110 nan
111 nan
112 nan
113 nan
114 nan
115 nan
116 nan
117 nan
118 nan
119 nan
120 nan
121 nan
122 nan
123 nan
124 nan
125 nan
126 nan
127 nan
128 nan
129 nan
130 nan
131 nan
132 nan
133 nan
134 nan
135 nan
